# 线性回归

本章内容包括：

1. 多元回归
2. 工具变量法
3. 混合截面数据和简单DID
4. 面板数据

工具和数据：

1. 回归采用`linearmodels`包，https://bashtage.github.io/linearmodels/
2. 部分示例和数据集来自伍德里奇《计量经济学导论》，这个数据已经被做成的Python包`Wooldridge`

第一次运行之前，在任何一个Python单元格中执行以下两段代码，以安装上述2个包。注意开头有个`!`

`!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple linearmodels`

`!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple Wooldridge`

## 普通OLS多元回归

这里介绍最普通的多元线性回归的方法。采用的范例为《导论》中的例7.1，这个例子试图研究空气污染和房价之间的关系，采用的数据集是`hprice2`。



这里介绍最普通的多元线性回归的方法。采用的范例为《导论》中的例4.5，这个例子试图研究空气污染和房价之间的关系，采用的数据集是`hprice2`。

![](images/w4.5.png){width=800}

In [133]:
# 导入需要用到的模块
import numpy as np
import pandas as pd
import wooldridge
from statsmodels.api import add_constant

In [116]:
wooldridge.data("hprice2", description=True)  # 打印数据的信息
df = wooldridge.data("hprice2") # 导入数据到df
df = add_constant(df, has_constant="add")
df.head()

name of dataset: hprice2
no of variables: 12
no of observations: 506

+----------+-------------------------------+
| variable | label                         |
+----------+-------------------------------+
| price    | median housing price, $       |
| crime    | crimes committed per capita   |
| nox      | nit ox concen; parts per 100m |
| rooms    | avg number of rooms           |
| dist     | wght dist to 5 employ centers |
| radial   | access. index to rad. hghwys  |
| proptax  | property tax per $1000        |
| stratio  | average student-teacher ratio |
| lowstat  | perc of people 'lower status' |
| lprice   | log(price)                    |
| lnox     | log(nox)                      |
| lproptax | log(proptax)                  |
+----------+-------------------------------+

D. Harrison and D.L. Rubinfeld (1978), “Hedonic Housing Prices and the
Demand for Clean Air,” by Harrison, D. and D.L.Rubinfeld, Journal of
Environmental Economics and Management 5, 81-102. Diego Garcia, a
for

,const,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,1.0,24000.0,0.006,5.38,6.57,4.09,1,29.600000,15.300000,4.98,10.085809,1.682688,5.690360
1,1.0,21599.0,0.027,4.69,6.42,4.97,2,24.200001,17.799999,9.14,9.980402,1.545433,5.488938
2,1.0,34700.0,0.027,4.69,7.18,4.97,2,24.200001,17.799999,4.03,10.454495,1.545433,5.488938
3,1.0,33400.0,0.032,4.58,7.00,6.06,3,22.200001,18.700001,2.94,10.416311,1.521699,5.402678
4,1.0,36199.0,0.069,4.58,7.15,6.06,3,22.200001,18.700001,5.33,10.496787,1.521699,5.402678


注意，我们需要的变量，只有ldist不在数据集中。

In [120]:
df["ldist"] = np.log(df["dist"])

进行回归

In [132]:
from linearmodels import OLS

res_ols = OLS(df["lprice"], df[["const", "lnox", "ldist", "rooms", "stratio"]]).fit(
    cov_type="unadjusted"
)

print(res_ols)  # 打印一下回归结果

                            OLS Estimation Summary                            
Dep. Variable:                 lprice   R-squared:                      0.5840
Estimator:                        OLS   Adj. R-squared:                 0.5807
No. Observations:                 506   F-statistic:                    710.44
Date:                Fri, Apr 19 2024   P-value (F-stat)                0.0000
Time:                        18:02:05   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          11.084     0.3165     35.016     0.0000      10.463      11.704
lnox          -0.9535     0.1162    -8.2086     0.00

也可以采用公式版的写法

In [128]:
res_ols_f = OLS.from_formula("lprice ~ lnox + ldist + rooms + stratio", data=df).fit(
    cov_type="unadjusted"
)

print(res_ols_f)  

                            OLS Estimation Summary                            
Dep. Variable:                 lprice   R-squared:                      0.5840
Estimator:                        OLS   Adj. R-squared:                 0.5807
No. Observations:                 506   F-statistic:                    710.44
Date:                Fri, Apr 19 2024   P-value (F-stat)                0.0000
Time:                        18:00:38   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      11.084     0.3165     35.016     0.0000      10.463      11.704
lnox          -0.9535     0.1162    -8.2086     0.00



这里采用伍德里奇的《计量经济学导论》中第15章中的例子。

例 15.1，估计已婚女性的教育回报，这个例子中，作者想用已婚女性的受教育程度educ，来解释她的收入wage。





在linearmodels包中已经包括了这份数据，我们可以从包中读取。当然， 一般情况下我们会从csv，dta或者excel文件中读取数据，读取数据会在Pandas的章节介绍。

In [99]:
import numpy as np
import pandas as pd
import wooldridge
from statsmodels.api import add_constant

print(wooldridge.data("mroz", description=True))
df = wooldridge.data("mroz")

print(df.DESCR)  # 列出变量的描述
data = df.load()  # 读取数据
data = data.dropna()  # 去掉na
data = add_constant(data, has_constant="add")  # 添加常数项

name of dataset: mroz
no of variables: 22
no of observations: 753

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| inlf     | =1 if in lab frce, 1975         |
| hours    | hours worked, 1975              |
| kidslt6  | # kids < 6 years                |
| kidsge6  | # kids 6-18                     |
| age      | woman's age in yrs              |
| educ     | years of schooling              |
| wage     | est. wage from earn, hrs        |
| repwage  | rep. wage at interview in 1976  |
| hushrs   | hours worked by husband, 1975   |
| husage   | husband's age                   |
| huseduc  | husband's years of schooling    |
| huswage  | husband's hourly wage, 1975     |
| faminc   | family income, 1975             |
| mtr      | fed. marg. tax rte facing woman |
| motheduc | mother's years of schooling     |
| fatheduc | father's years of schooling     |
| unem     | unem. rate in county of res

AttributeError: 'DataFrame' object has no attribute 'DESCR'

首先我们要进行的OLS估计是：

$$\log(wage) = \beta_0 + \beta_1 educ + u$$

我们将会使用linearmodels中的OLS对象，初始化的时候接受几个参数，第一个是被解释变量，第二个是解释变量

1. 被解释变量/因变量是工资 `data['wage']`，但是这里要取对数 `np.log(data['wage'])`
2. 解释变量/自变量是教育程度 `data['edu']`，要加上常数项，即`data[["const", "educ"]]`。注意常数项是前面`add_constant`函数添加的。

一般的用法就是：

回归结果 = OLS(因变量, 自变量).fit()


In [ ]:
from linearmodels import OLS

res_ols = OLS(np.log(data["wage"]), data[["const", "educ"]]).fit()

print(res_ols)  # 打印一下回归结果

可见，教育educ的系数是0.1086，每接受多1年教育，可以获得11%的回报。P-value约等于0，非常显著。这个结果符合《导论》保持一致。

除了你直接指定数据列，`np.log(data["wage"]), data[["const", "educ"]]`，还可以用`OLS.from_formula()`指定回归公式。

这里的公式是一个字符串，一般性的写法是 "被解释变量 ~ 解释变量1 + 解释变量2 + .... "，并且支持常用的运算，比如取对数。默认情况下会自带常数项，因此公式里不用特别指定。

那么，实现上述回归，公式就是 `"log(wage) ~ educ"`
一般的用法是`OLS.from_formula(公式， data = 数据).fit()`

In [ ]:
res_ols2 = OLS.from_formula("log(wage) ~ educ", data=data).fit()
print(res_ols2)

可以对比上述两种方法，得到的结果是一样的。除了你肉眼看，还可以用compare函数来比较回归。

In [ ]:
from linearmodels.iv.results import compare

print(compare([res_ols, res_ols2]))

可见educ的系数是一样的，证明OLS()和OLS.from_formula() 本质上相同，看你习惯用哪个。

注意，add_constant()函数生成的常数项的名称是const，但是from_formula生成的常数项叫Intercept（截距），但只是不同的包作者习惯用不同的名称，本质上这两者是一样的。

多元的版本也是类似，比如加入工作经验exper及其平方项作为控制变量

$$\log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3 expersq +  u$$


In [ ]:
from linearmodels import OLS

res_ols = OLS(np.log(data["wage"]), data[["const", "educ", "exper", "expersq"]]).fit()

print(res_ols) 

In [ ]:
from linearmodels import OLS

res_ols2 = OLS.from_formula("log(wage) ~ educ + exper + expersq", data=data).fit()

print(res_ols2)

In [ ]:
print(compare([res_ols, res_ols2]))

## 工具变量法

继续前面的例子。我们使用父亲的受教育程度 fatheduc 作为 educ 的工具变量。

工具变量回归要导入IV2SLS，用法是 `IV2SLS(因变量, 外生变量, 内生变量, 工具变量)`，更通俗的表达是 `IV2SLS(因变量, 控制变量, 自变量, 工具变量)`

在本案中，则是`IV2SLS(np.log(data.wage), data[["const"]], data.educ, data.fathedcu)`

`fit()`函数可以指定方差的计算方式，我们这里不展开。

In [ ]:
from linearmodels.iv import IV2SLS

res_second = IV2SLS(np.log(data.wage), data[["const"]], data.educ, data.fatheduc).fit(
    cov_type="unadjusted"  # 使用传统的OLS方差估计,假设误差项是同方差的
)
print(res_second)

采用公式版，公式的写法是 `dep ~ exog + [ endog ~ instruments]`

In [ ]:
res_second2 = IV2SLS.from_formula("log(wage) ~ [educ ~ fatheduc]", data=data).fit(
    cov_type="unadjusted"
)
print(res_second)

当然两种方法的结果也是一样的，看你习惯用哪个。

比较OLS和2SLS的回归结果

In [ ]:
print(compare([res_ols, res_second]))

2SLS的估计值为5.9%，是OLS估计的一半左右

## 稍微复杂一点的例子

例子来自 《导论》 第15章的例15.4，作者试图用男性样本中的工资和受教育程度 `educ` 来估计教育回报 `wage` ，并且用一个人是否在一所四年制大学附近成长 `nearc4` ，来自作为受教育程度的工具变量。还包括其他控制变量，等等。


同样，希望回答的问题也是“多读一年书能多拿多少钱工资？”

数据是`datasets.card`，可以看到我们需要的变量都在里面。


In [ ]:
from linearmodels.datasets import card

data = card.load()
print(card.DESCR)
data = add_constant(data)

我们首先确定回归用的数据

In [ ]:
dep = ["wage"]  # 因变量：工资
endog = ["educ"]  # 自变量：教育程度
exog = [  # 外生变量
    "const",
    "exper",  # 工作经验
    "expersq",  # 工作经验的平方
    "black",  # 是否是黑人
    "smsa",  # 是否住在大城市及其郊区
    "south",  # 是否住在南方
    "smsa66",  # 66年时候居住在何处
    "reg662",  # 一系列地区虚拟变量
    "reg663",
    "reg664",
    "reg665",
    "reg666",
    "reg667",
    "reg668",
    "reg669",
]
instr = ["nearc4"]  # 工具变量：是否在4年制大学附近成长
data = data[dep + exog + endog + instr].dropna()  # 把上述变量集合到data中
data.head()

### OLS

先做一个OLS, 直接用educ来解释wage。

 `log(wage) ~ educ + controls`


当我们不指定内生变量和工具变量的时候，IV2SLS()和OLS()结果一样

In [ ]:
res = IV2SLS(np.log(data.wage), data[exog + endog], None, None).fit()
print(res)

educ的系数是7%左右，每多读书1年，工资增加7%。

### 2SLS

利用nearc4来做educ的工具变量.

一阶段是: educ ~ nearc4 + controls

二阶段是: 

我们首先手动做两阶段回归，然后再用一条命令完成，并从中提取第一阶段。

一阶段：

In [ ]:
res = IV2SLS(data.educ, data[instr + exog], None, None).fit(
    cov_type="unadjusted"
)  # 不指定内生变量和工具变量，最后2个参数都是None
print(res)

nearc4的系数很显著，上面的结果同《导论》的公式15.32(p495)。



In [ ]:
res = IV2SLS(np.log(data.wage), data[exog + endog], None, None).fit()
print(res)

In [ ]:
res = IV2SLS(np.log(data.wage), data[exog + endog], None, None).fit()
print(res)

In [ ]:
res_2sls = IV2SLS(np.log(data.wage), data[exog], data[endog], data[instr]).fit()
res_2sls

公式版

In [ ]:
import numpy as np

formula = (
    "np.log(wage) ~ 1 + exper + expersq + black + smsa + south + smsa66 + reg662 + reg663 + reg664 + "
    "reg665 + reg666 + reg667 + reg668 + reg669 + [educ ~ nearc4]"
)
mod = IV2SLS.from_formula(formula, data)
res_formula = mod.fit(cov_type="unadjusted")
res_formula

## 混合截面数据和简单DID

独立混合截面数据(pooled)：指从在不同的年份，从一个大的整体中进行独立抽样形成的数据，比如人口普查。两次抽样之间，被选中的个体很可能不同。

### 年度虚拟变量


最基本的方法是利用年度虚拟变量，即我们允许不同的年份有不同的截距。


In [96]:
df = pd.read_csv("https://www.meyerperin.com/econometrics-python/data/fertil1.csv")
df.head()

,year,educ,meduc,feduc,age,kids,black,east,northcen,west,...,y80,y82,y84,agesq,y74educ,y76educ,y78educ,y80educ,y82educ,y84educ
0,72,12,8,8,48,4,0,0,1,0,...,0,0,0,2304,0,0,0,0,0,0
1,72,17,8,18,46,3,0,0,0,0,...,0,0,0,2116,0,0,0,0,0,0
2,72,12,7,8,53,2,0,0,1,0,...,0,0,0,2809,0,0,0,0,0,0
3,72,12,12,10,42,2,0,0,1,0,...,0,0,0,1764,0,0,0,0,0,0
4,72,12,3,8,51,2,0,0,0,0,...,0,0,0,2601,0,0,0,0,0,0


In [97]:
m1 = OLS.from_formula("kids ~ educ", data=df).fit()
print(m1)

                            OLS Estimation Summary                            
Dep. Variable:                   kids   R-squared:                      0.0498
Estimator:                        OLS   Adj. R-squared:                 0.0489
No. Observations:                1129   F-statistic:                    43.960
Date:                Fri, Apr 19 2024   P-value (F-stat)                0.0000
Time:                        17:01:35   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.5163     0.2763     16.345     0.0000      3.9748      5.0579
educ          -0.1397     0.0211    -6.6302     0.00


### 兴建焚化炉对房价的影响

《导论》 例13.3，计算兴建焚化炉对住房价格的影响

In [93]:
df = pd.read_csv("https://www.meyerperin.com/econometrics-python/data/kielmc.csv")
df.head()

,year,age,agesq,nbh,cbd,intst,lintst,price,rooms,area,...,lprice,y81,larea,lland,y81ldist,lintstsq,nearinc,y81nrinc,rprice,lrprice
0,1978,48,2304.0,4,3000.0,1000.0,6.9078,60000.0,7,1660,...,11.002100,0,7.414573,8.429017,0.0,47.717705,1,0,60000.0,11.002100
1,1978,83,6889.0,4,4000.0,1000.0,6.9078,40000.0,6,2612,...,10.596635,0,7.867871,9.032409,0.0,47.717705,1,0,40000.0,10.596635
2,1978,58,3364.0,4,4000.0,1000.0,6.9078,34000.0,6,1144,...,10.434115,0,7.042286,8.517193,0.0,47.717705,1,0,34000.0,10.434115
3,1978,11,121.0,4,4000.0,1000.0,6.9078,63900.0,5,1136,...,11.065075,0,7.035269,9.210340,0.0,47.717705,1,0,63900.0,11.065075
4,1978,48,2304.0,4,4000.0,2000.0,7.6009,44000.0,5,1868,...,10.691945,0,7.532624,9.210340,0.0,57.773682,1,0,44000.0,10.691945


In [ ]:
pd.read_csv("https://www.meyerperin.com/econometrics-python/data/kielmc_info.csv")

只使用81年的数据，进行一个天真的回归

In [ ]:
m1 = OLS.from_formula("rprice ~ nearinc", data=df.query("y81==1")).fit(
    cov_type="unadjusted"
)
print(m1)

可见，越接近焚化炉，房屋售价越低。这是否证明房屋被焚化炉影响了？

我们对81年之前的数据进行回归

In [ ]:
m2 = OLS.from_formula("rprice ~ nearinc", data=df.query("y81==0")).fit(
    cov_type="unadjusted"
)
print(m2)

可见没有焚化炉传闻时，这个地方的房价本来就低于周边。或者说，焚化炉可能故意选择一个房价最低的地方来建立。

那么如何识别焚化炉对这个地区放假的影响？

做个简单的DID，复原《导论》的表13.2

In [ ]:
m3 = OLS.from_formula("rprice ~ y81 + nearinc + y81*nearinc", data=df).fit(
    cov_type="unadjusted"
)
# print(m3)

In [ ]:
m4 = OLS.from_formula(
    "rprice ~ y81 + nearinc + y81*nearinc + age + agesq", data=df
).fit(cov_type="unadjusted")
# print(m4)

In [ ]:
m5 = OLS.from_formula(
    "rprice ~ y81 + nearinc + y81*nearinc + age + agesq + intst + land + area + rooms + baths",
    data=df,
).fit(cov_type="unadjusted")
#  print(m5)

和原表13.2有轻微出入，但没有实质性差异

In [ ]:
print(compare([m3, m4, m5], stars=True))

## 面板数据

面板数据：对同一组个体进行多期的观察。使用面板数据可以处理不随时间变化的非观测相应（即可以切掉一些遗漏变量：不随时间变化的、不可观测的个体特征）。

### 差分法








In [94]:
df = pd.read_csv("https://www.meyerperin.com/econometrics-python/data/crime2.csv")
df.head()

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clcrimes,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,NaN,0.350872,NaN,NaN,NaN,NaN,NaN,NaN
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,0.009871,0.072614,-0.062743,0.262802,-0.08807,0.977952,-4.5,0.072615,4.312912,-4.540268
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,NaN,0.688772,NaN,NaN,NaN,NaN,NaN,NaN
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,0.104170,0.136568,-0.032398,0.658612,-0.03016,0.200762,-2.7,0.136568,4.531899,-2.962654
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,NaN,0.523614,NaN,NaN,NaN,NaN,NaN,NaN


### 其他例子

In [ ]:
import pandas as pd
from linearmodels.datasets import wage_panel

data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(["nr", "year"])
data["year"] = year
print(wage_panel.DESCR)
data.head()

In [ ]:
data.info()

不考虑个体相应（不管个体在不同的年份是否相同），我们可以进行混合截面回归， PooledOLS

In [ ]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

In [ ]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

固定效应模型


In [ ]:
from linearmodels.panel import PanelOLS

exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

In [ ]:
exog_vars = ["expersq", "union", "married"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, time_effects=True)
fe_te_res = mod.fit()
print(fe_te_res)

In [ ]:
from linearmodels.panel.data import PanelData

exog_vars = ["expersq", "union", "married"]
exog = sm.add_constant(data[exog_vars])
# Use the `PanelData` structure to simplify constructing the time IDs
time_ids = pd.DataFrame(
    PanelData(data.lwage).time_ids, index=data.index, columns=["Other Effect"]
)

mod = PanelOLS(data.lwage, exog, entity_effects=True, other_effects=time_ids)
fe_oe_res = mod.fit()
print(fe_oe_res)

In [ ]:
from statsmodels.datasets import grunfeld

data = grunfeld.load_pandas().data
data.head()

In [ ]:
data = data.set_index(["firm", "year"])
data.head()

In [ ]:
from linearmodels import PanelOLS

mod = PanelOLS.from_formula("invest ~ value + capital + EntityEffects", data=data)
print(mod.fit())

In [ ]:
mod = PanelOLS.from_formula(
    "invest ~ 1 + value + capital + EntityEffects + TimeEffects", data=data
)
print(mod.fit())